Fullname: qtf

Student ID: 69

# HW3: CUDA Memory

**To compile your file, you can use this command:** \
`nvcc filename.cu -o execute_filename` \
***You can use Vietnamese to anwser the questions***

You can add/delete cells as you see fit. Just don't delete the cells with the Teacher's words.

---

Below is the command to install `Nsight system` on Colab. You just need to run the cell.

In [ ]:
# Here is the command to install Nsight system on Colab. You just need to run the cell.
!apt-get update
!apt-get install -y cuda-nsight-systems-12-2

Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

- The above installation will be remove when colab restart its runtime. It should be fine for this homework because we don't use Nsight system that much.
- But if you want to persistent install Nsight system, you mount space from your Google Drive to your Colab VM. Read [Here](https://stackoverflow.com/questions/76784746/how-to-use-nsys-in-google-colab) for guidelines how to do thhis.

---

In [1]:
from numba import cuda

major, minor = cuda.get_current_device().compute_capability
print(f"GPU compute capability: {major}.{minor}")

GPU compute capability: 7.5


In [2]:
from google.colab import files

uploaded = files.upload()

Saving HW3_P1.cu to HW3_P1.cu
Saving HW3_P2.cu to HW3_P2.cu
Saving in.pnm to in.pnm
Saving test_1x1_pixel.pnm to test_1x1_pixel.pnm
Saving test_1x50_row.pnm to test_1x50_row.pnm
Saving test_250x250.pnm to test_250x250.pnm


## Exercises 1

### Thử nghiệm với ảnh mẫu (512x512)

In [3]:
!nvcc -arch=sm_{major}{minor} HW3_P1.cu -o HW3_P1

In [4]:
!./HW3_P1 in.pnm out.pnm

**********GPU info**********
Name: Tesla T4
Compute capability: 7.5
Num SMs: 40
Max num threads per SM: 1024
Max num warps per SM: 32
GMEM: 15828320256 bytes
CMEM: 65536 bytes
L2 cache: 4194304 bytes
SMEM / one SM: 65536 bytes
****************************

Image size (width x height): 512 x 512

Kernel 1, block size 16x16, grid size 32x32
Kernel time: 0.765792 ms
Error: 0.000703

Kernel 2, block size 16x16, grid size 32x32
Kernel time: 0.341760 ms
Error: 0.000703

Kernel 3, block size 16x16, grid size 32x32
Kernel time: 0.268256 ms
Error: 0.000703


**Nhận xét**

| Version | Logic | Pros | Kernel time |
|-----------|----------|-----------|-------------|
| Kernel 1 | Đọc trực tiếp Global Memory | Dễ hiểu, không cần tile | 0.766 ms |
| Kernel 2 | Shared Memory tile | Giảm số lần truy cập GMEM | 0.342 ms |
| Kernel 3 | Shared + Constant Memory | Broadcast hệ số filter, tái sử dụng tile | 0.268 ms |

- **Shared Memory** giảm thời gian vì mỗi block nạp tile + halo lên SMEM một lần rồi tái dùng cho 81 phép nhân (filterWidth=9), thay vì mỗi thread tự đọc 81 lần từ GMEM; chi phí `__syncthreads()` nhỏ hơn rất nhiều so với lượng truy cập bộ nhớ tiết kiệm được.
- **Constant Memory** ở kernel 3 giúp warp nhận hệ số filter qua broadcast một giao dịch, loại bỏ thêm 32 lượt đọc GMEM cho mỗi hệ số, nên tiếp tục rút ~20% thời gian.

### Thử nghiệm đặc biệt

In [5]:
!./HW3_P1 test_1x1_pixel.pnm out_test_1x1_pixel.pnm

**********GPU info**********
Name: Tesla T4
Compute capability: 7.5
Num SMs: 40
Max num threads per SM: 1024
Max num warps per SM: 32
GMEM: 15828320256 bytes
CMEM: 65536 bytes
L2 cache: 4194304 bytes
SMEM / one SM: 65536 bytes
****************************

Image size (width x height): 1 x 1

Kernel 1, block size 16x16, grid size 1x1
Kernel time: 0.301184 ms
Error: 0.000000

Kernel 2, block size 16x16, grid size 1x1
Kernel time: 0.065568 ms
Error: 0.000000

Kernel 3, block size 16x16, grid size 1x1
Kernel time: 0.060864 ms
Error: 0.000000

Kernel 4, block size 1x1, grid size 1x1
Kernel time: 0.052704 ms
Error: 0.666667


In [6]:
!./HW3_P1 test_1x50_row.pnm out_test_1x50_row.pnm

**********GPU info**********
Name: Tesla T4
Compute capability: 7.5
Num SMs: 40
Max num threads per SM: 1024
Max num warps per SM: 32
GMEM: 15828320256 bytes
CMEM: 65536 bytes
L2 cache: 4194304 bytes
SMEM / one SM: 65536 bytes
****************************

Image size (width x height): 50 x 1

Kernel 1, block size 16x16, grid size 4x1
Kernel time: 0.279584 ms
Error: 0.006667

Kernel 2, block size 16x16, grid size 4x1
Kernel time: 0.083872 ms
Error: 0.006667

Kernel 3, block size 16x16, grid size 4x1
Kernel time: 0.059968 ms
Error: 0.006667

Kernel 4, block size 50x1, grid size 1x1
Kernel time: 0.042208 ms
Error: 0.060000


In [7]:
!./HW3_P1 test_250x250.pnm out_test_250x250.pnm

**********GPU info**********
Name: Tesla T4
Compute capability: 7.5
Num SMs: 40
Max num threads per SM: 1024
Max num warps per SM: 32
GMEM: 15828320256 bytes
CMEM: 65536 bytes
L2 cache: 4194304 bytes
SMEM / one SM: 65536 bytes
****************************

Image size (width x height): 250 x 250

Kernel 1, block size 16x16, grid size 16x16
Kernel time: 0.385312 ms
Error: 0.000235

Kernel 2, block size 16x16, grid size 16x16
Kernel time: 0.131904 ms
Error: 0.000235

Kernel 3, block size 16x16, grid size 16x16
Kernel time: 0.114688 ms
Error: 0.000235


**Nhận xét**

| Ảnh đầu vào      | Kernel 1 (GMEM-only) | Kernel 2 (Tile SMEM) | Kernel 3 (SMEM + CMEM) | Kernel 4 (Line-SMEM) |
|------------------|----------------------|----------------------|------------------------|----------------------|
| `test_1x1_pixel` | 0.301 ms / 0.00000   | 0.066 ms / 0.00000   | 0.061 ms / 0.00000     | 0.053 ms / 0.66667   |
| `test_1x50_row`  | 0.280 ms / 0.00667   | 0.084 ms / 0.00667   | 0.060 ms / 0.00667     | 0.042 ms / 0.06000   |
| `test_250x250`   | 0.385 ms / 0.00024   | 0.132 ms / 0.00024   | 0.115 ms / 0.00024     | —                    |

- Kernel 4 chuyên cho ảnh 1D nên nhanh nhất, nhưng sai số lớn (thấy rõ ở `test_1x1_pixel`, `test_1x50_row`). Khi ảnh vuông, Kernel 3 vẫn là lựa chọn cân bằng giữa tốc độ và độ chính xác.

## Exercises 2

In [8]:
!nvcc -arch=sm_{major}{minor} HW3_P2.cu -o HW3_P2

In [9]:
!./HW3_P2

╔════════════════════════════════════════╗
║  VECTOR OPERATIONS PIPELINE EXERCISE   ║
╚════════════════════════════════════════╝

Configuration:
  Number of vectors: 8
  Elements per vector: 4194304 (16.00 MB)
  Total data: 128.00 MB

GPU: Tesla T4
Concurrent kernels supported: Yes
Number of copy engines: 3


=== SEQUENTIAL (NO STREAMS) ===
Time: 25.53 ms
Breakdown (avg per vector) -> H2D: 1.41 ms (44.6%), Kernels: 0.46 ms (14.5%), D2H: 1.29 ms (40.9%)

=== BREADTH-FIRST PIPELINE ===
Time: 16.97 ms
Breakdown (avg per vector) -> H2D: 7.42 ms (69.2%), Kernels: 0.81 ms (7.5%), D2H: 2.49 ms (23.2%)

=== DEPTH-FIRST PIPELINE ===
Time: 15.43 ms
Breakdown (avg per vector) -> H2D: 7.46 ms (76.9%), Kernels: 0.47 ms (4.8%), D2H: 1.77 ms (18.2%)

=== VERIFICATION ===
✓ All results match!

╔════════════════════════════════════════╗
║         PERFORMANCE SUMMARY            ║
╚════════════════════════════════════════╝
Sequential:     25.53 ms
Breadth-First:  16.97 ms  (1.50x speedup)
Depth-First:   

In [10]:
!./HW3_P2 16

╔════════════════════════════════════════╗
║  VECTOR OPERATIONS PIPELINE EXERCISE   ║
╚════════════════════════════════════════╝

Configuration:
  Number of vectors: 16
  Elements per vector: 4194304 (16.00 MB)
  Total data: 256.00 MB

GPU: Tesla T4
Concurrent kernels supported: Yes
Number of copy engines: 3


=== SEQUENTIAL (NO STREAMS) ===
Time: 50.25 ms
Breakdown (avg per vector) -> H2D: 1.38 ms (44.2%), Kernels: 0.45 ms (14.4%), D2H: 1.29 ms (41.4%)

=== BREADTH-FIRST PIPELINE ===
Time: 35.45 ms
Breakdown (avg per vector) -> H2D: 14.84 ms (74.1%), Kernels: 0.95 ms (4.8%), D2H: 4.25 ms (21.2%)

=== DEPTH-FIRST PIPELINE ===
Time: 30.66 ms
Breakdown (avg per vector) -> H2D: 8.32 ms (77.9%), Kernels: 0.46 ms (4.3%), D2H: 1.91 ms (17.8%)

=== VERIFICATION ===
✓ All results match!

╔════════════════════════════════════════╗
║         PERFORMANCE SUMMARY            ║
╚════════════════════════════════════════╝
Sequential:     50.25 ms
Breadth-First:  35.45 ms  (1.42x speedup)
Depth-First: 

In [11]:
!./HW3_P2 32

╔════════════════════════════════════════╗
║  VECTOR OPERATIONS PIPELINE EXERCISE   ║
╚════════════════════════════════════════╝

Configuration:
  Number of vectors: 32
  Elements per vector: 4194304 (16.00 MB)
  Total data: 512.00 MB

GPU: Tesla T4
Concurrent kernels supported: Yes
Number of copy engines: 3


=== SEQUENTIAL (NO STREAMS) ===
Time: 100.56 ms
Breakdown (avg per vector) -> H2D: 1.38 ms (44.3%), Kernels: 0.44 ms (14.3%), D2H: 1.29 ms (41.4%)

=== BREADTH-FIRST PIPELINE ===
Time: 72.80 ms
Breakdown (avg per vector) -> H2D: 29.96 ms (76.9%), Kernels: 1.19 ms (3.1%), D2H: 7.83 ms (20.1%)

=== DEPTH-FIRST PIPELINE ===
Time: 62.12 ms
Breakdown (avg per vector) -> H2D: 8.90 ms (78.5%), Kernels: 0.46 ms (4.0%), D2H: 1.99 ms (17.5%)

=== VERIFICATION ===
✓ All results match!

╔════════════════════════════════════════╗
║         PERFORMANCE SUMMARY            ║
╚════════════════════════════════════════╝
Sequential:     100.56 ms
Breadth-First:  72.80 ms  (1.38x speedup)
Depth-First

In [ ]:
# Generate report with nsight system
!nsys profile ./HW3_P2

### Report

#### Question 1: Performance Analysis
**Tại sao depth-first nhanh hơn breadth-first?**

**Depth-first** pipeline mỗi stream thực thi tuần tự đầy đủ: H2D → Scale → Add → Square → D2H. Điều này cho phép:
- **Overlapping tối đa**: Khi stream 1 đang compute (kernels), stream 2 có thể H2D, stream 3 D2H → copy engines và SM hoạt động song song liên tục.
- **Giảm idle time**: GPU không phải chờ đợi giữa các phase như breadth-first.

**Breadth-first** thực thi theo batch từng loại operation (tất cả H2D → tất cả kernels → tất cả D2H), dẫn đến:
- Copy engines rảnh trong phase compute, SM rảnh trong phase transfer.
- Overhead đồng bộ giữa các phase.

**Kết quả thực nghiệm**: Với 16 vectors, depth-first (30.66 ms) nhanh hơn breadth-first (35.45 ms) ~16%.

#### Question 2: Scalability
| Vectors | Sequential (ms) | Breadth-First (ms) | Depth-First (ms) | Speedup BF | Speedup DF |
|---------|-----------------|--------------------|-----------------|-----------:|----------:|
| 8       | 25.53           | 16.97              | 15.43           | 1.50×      | 1.65×     |
| 16      | 50.25           | 35.45              | 30.66           | 1.42×      | 1.64×     |
| 32      | 100.56          | 72.80              | 62.12           | 1.38×      | 1.62×     |

**Nhận xét**:
- Thời gian tăng gần tuyến tính theo số vectors do tổng data tăng (128 MB → 512 MB).
- Depth-first duy trì lợi thế 15-17% so với breadth-first nhờ pipeline hiệu quả hơn.
- Speedup giảm dần khi scale up do GPU đạt gần saturation bandwidth.

![](output.png)

![](output2.png)

#### Question 3: Memory Transfer vs Computation
**8 vectors (Sequential)**:
- H2D: 1.41 ms (44.6%)
- Kernels: 0.46 ms (14.5%)
- D2H: 1.29 ms (40.9%)
- **→ 85% thời gian là memory transfer**, compute chỉ chiếm 15%.

**8 vectors (Breadth-First)**: 
- Streams giúp overlap, giảm total time từ 25.53 ms → 16.97 ms (1.50× speedup).
- Transfer vẫn chiếm ~92% (H2D 69.2% + D2H 23.2%).

**32 vectors (Depth-First)**:
- H2D: 78.5%, Kernels: 4.0%, D2H: 17.5%
- Depth-first che giấu compute bằng cách overlap với transfer của streams khác.
- **Bottleneck chính**: PCIe bandwidth, không phải SM throughput.

**Kết luận**: Workload này memory-bound. Streams giúp tận dụng tối đa băng thông nhưng không thể vượt qua giới hạn vật lý của PCIe.

#### Question 4

![Nsight](NVIDIA_Nsight_Systems.png)